System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 6 on 6 virtual cores
Environment:
  JULIA_EDITOR = code
  JULIA_NUM_THREADS = 6


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/UCLA_files/course_work/BIS_M257/hw/biostat-257-2023-spring/hw7`


Status `~/Documents/UCLA_files/course_work/BIS_M257/hw/biostat-257-2023-spring/hw7/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [31c24e10] Distributions v0.25.95
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


Again we continue with the linear mixed effects model (LMM)
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T.
$$
Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2).
$$

In HW6, we used the nonlinear programming (NLP) approach (Newton type algorithms) for optimization. In this assignment, we derive and implement an expectation-maximization (EM) algorithm for the same problem.

In [3]:
# load necessary packages; make sure install them first
using BenchmarkTools, Distributions, LinearAlgebra, Random

## Q1. (10 pts) Refresher on normal-normal model

Assume the conditional distribution
$$
\mathbf{y} \mid \boldsymbol{\gamma} \sim N(\mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \boldsymbol{\gamma}, \sigma^2 \mathbf{I}_n)
$$
and the prior distribution
$$
\boldsymbol{\gamma} \sim N(\mathbf{0}_q, \boldsymbol{\Sigma}).
$$
By the Bayes theorem, the posterior distribution is
\begin{align*}
f(\boldsymbol{\gamma} \mid \mathbf{y}) &=& \frac{f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma})}{f(\mathbf{y})} 
\end{align*}
where $f$ denotes corresponding density. 

Show that the posterior distribution of random effects $\boldsymbol{\gamma}$ is a multivariate normal with mean
\begin{align*}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &= \sigma^{-2} (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1 } \mathbf{Z}^T (\mathbf{y} - \mathbf{X} \boldsymbol{\beta}) \\
&= \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} (\mathbf{y} - \mathbf{X} \boldsymbol{\beta})
\end{align*}
and covariance
$$
\begin{align}
\text{Var} (\boldsymbol{\gamma} \mid \mathbf{y}) &= (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1} \\
&= \boldsymbol{\Sigma} - \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} \mathbf{Z} \boldsymbol{\Sigma}.
\end{align}
$$

### Q1 Answer
#### Part 1
Considering that both $\mathbf{y}$ and $\boldsymbol{\gamma}$ are multivariate normal, the joint distribution of $\mathbf{y}$ and $\boldsymbol{\gamma}$ is also multivariate normal. Therefore, we have the following conclusion for the conditional distribution of $\boldsymbol{\gamma}$ given $\mathbf{y}$:
$$
\begin{aligned}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &= 
\mathbb{E} (\boldsymbol{\gamma}) + \operatorname{Cov}(\boldsymbol{\gamma}, \mathbf{y}) [\operatorname{Var}(\mathbf{y})]^{-1}[\mathbf{y} - \mathbb{E}(\mathbf{y})]\\
\text{And we have:}\quad \mathbb{E} (\boldsymbol{\gamma}) &= \mathbf{0}_q\\
\mathbb{E} (\mathbf{y}) &= \mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \mathbb{E}(\boldsymbol{\gamma}) = \mathbf{X} \boldsymbol{\beta}\\
\operatorname{Var}(\mathbf{y}) &= \mathbf{Z}\mathbf{\Sigma}\mathbf{Z}^T + \sigma^2 \mathbf{I}_n \\
\operatorname{Cov}(\boldsymbol{\gamma}, \mathbf{y}) &= \mathbb{E}\left[(\boldsymbol{\gamma} -\mathbb{E}(\boldsymbol{\gamma}))((\boldsymbol{y} -\mathbb{E}(\boldsymbol{y}))^T\right] \\
&= \mathbb{E}\left[\boldsymbol{\gamma}(\boldsymbol{y} -\mathbb{E}(\boldsymbol{y}))^T\right] \\
&= \mathbb{E}(\boldsymbol{\gamma} \boldsymbol{y}^T) - \mathbb{E}(\boldsymbol{\gamma})(\mathbf{X}\boldsymbol{\beta})^T \\
&= \mathbb{E}(\boldsymbol{\gamma} \boldsymbol{y}^T) \\
&= \mathbb{E}\left[\boldsymbol{\gamma} \mathbb{E}(\boldsymbol{y} \mid \boldsymbol{\gamma})^T\right] \\
&= \mathbb{E}(\boldsymbol{\gamma} (\mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \boldsymbol{\gamma})^T) \\
&= \mathbb{E}(\boldsymbol{\gamma}\boldsymbol{\gamma}^T\mathbf{Z}^T) \\
&= \mathbf{\Sigma}\mathbf{Z}^T \\
\text{Therefore}\quad \mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &= \mathbf{0}_q + \mathbf{\Sigma}\mathbf{Z}^T (\mathbf{Z}\mathbf{\Sigma}\mathbf{Z}^T + \sigma^2 \mathbf{I}_n)^{-1}(\mathbf{y} - \mathbf{X} \boldsymbol{\beta}) \\
&= \mathbf{\Sigma}\mathbf{Z}^T (\mathbf{Z}\mathbf{\Sigma}\mathbf{Z}^T + \sigma^2 \mathbf{I}_n)^{-1}(\mathbf{y} - \mathbf{X} \boldsymbol{\beta})
\end{aligned}
$$

#### Part 2
$$
\begin{aligned}
\operatorname{Var}(\boldsymbol{\gamma}\mid \mathbf{y}) &= \operatorname{Var}(\boldsymbol{\gamma}) - \operatorname{Cov}(\boldsymbol{\gamma}, \mathbf{y}) [\operatorname{Var}(\mathbf{y})]^{-1}\operatorname{Cov}(\boldsymbol{\gamma}, \mathbf{y})^T \\
&= \mathbf{\Sigma} - \mathbf{\Sigma}\mathbf{Z}^T (\mathbf{Z}\mathbf{\Sigma}\mathbf{Z}^T + \sigma^2 \mathbf{I}_n)^{-1}\mathbf{Z}\mathbf{\Sigma} \\
\end{aligned}
$$

## Q2. (20 pts) Derive EM algorithm

1. Write down the complete log-likelihood
$$
\sum_{i=1}^m \log f(\mathbf{y}_i, \boldsymbol{\gamma}_i \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)
$$

2. Derive the $Q$ function (E-step).
$$
Q(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2 \mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}).
$$

3. Derive the EM (or ECM) update of $\boldsymbol{\beta}$, $\boldsymbol{\Sigma}$, and $\sigma^2$ (M-step). 

### Q2 Answer
#### Part 1
Considering that $\log f(\mathbf{y}_i, \boldsymbol{\gamma}_i \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$, the following steps omit the subscript $i$ for simplicity:
$$
\begin{aligned}
\log f(\mathbf{y}, \boldsymbol{\gamma} \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) =& \log \left[ f(\mathbf{y} \mid \boldsymbol{\gamma}, \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)f( \boldsymbol{\gamma}\mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) \right] \\
=& \log  \left[f(\mathbf{y} \mid \boldsymbol{\beta}, \boldsymbol{\gamma}, \sigma^2) f(\boldsymbol{\gamma}\mid \boldsymbol{\Sigma})\right] \\
=& \log f(\mathbf{y} \mid \boldsymbol{\beta}, \boldsymbol{\gamma}, \sigma^2) + \log f(\boldsymbol{\gamma}\mid \boldsymbol{\Sigma}) \\
=& -\frac{n}{2}\log(2\pi) -\frac{1}{2}\log\det(\sigma^2\mathbf{I_n}) -\frac{1}{2}(\mathbf{y}-\mathbf{X}\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma})^T(\sigma^2\mathbf{I_n})^{-1}(\mathbf{y}-\mathbf{X}\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma})\\
&+ -\frac{q}{2}\log(2\pi) -\frac{1}{2}\log\det(\mathbf{\Sigma}) -\frac{1}{2}\boldsymbol{\gamma}^T\mathbf{\Sigma}^{-1}\boldsymbol{\gamma} \\
=& -\frac{n}{2}\log(2\pi) -\frac{n}{2}\log(\sigma^2) -\frac{1}{2\sigma^2}(\mathbf{y}-\mathbf{X}\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma})^T(\mathbf{y}-\mathbf{X}\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma})\\
& -\frac{q}{2}\log(2\pi) -\frac{1}{2}\log\det(\mathbf{\Sigma}) -\frac{1}{2}\boldsymbol{\gamma}^T\mathbf{\Sigma}^{-1}\boldsymbol{\gamma} \\
\end{aligned}
$$

Get it summed over the entire data set (i.e., summing over $i$):

$$
\begin{aligned}
\sum_{i=1}^{m}\log f(\mathbf{y}_i, \boldsymbol{\gamma}_i \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)
=& -\frac{\sum_{i=1}^{m} n_i}{2}\log(2\pi) -\frac{\sum_{i=1}^{m} n_i}{2}\log(\sigma^2) -\frac{1}{2\sigma^2}\sum_{i=1}^{m}(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma}_i)^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma}_i)\\
& -\frac{qm}{2}\log(2\pi) -\frac{m}{2}\log\det(\mathbf{\Sigma}) -\frac{1}{2}\sum_{i=1}^{m}\boldsymbol{\gamma}_i^T\mathbf{\Sigma}^{-1}\boldsymbol{\gamma}_i \\
\end{aligned}
$$

#### Part 2
$$
\begin{aligned}
& Q(\boldsymbol{\theta}|\boldsymbol{\theta}^{(t)}) \\
=&\mathbb{E}_{\boldsymbol{\gamma}|\mathbf{y},\boldsymbol{\theta}^{(t)}}\left[\log f(\mathbf{y}, \boldsymbol{\gamma} \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)\right] \\
=& -\frac{\sum_{i=1}^{m} n_i}{2}\log(2\pi) -\frac{\sum_{i=1}^{m} n_i}{2}\log(\sigma^2) -\frac{qm}{2}\log(2\pi) -\frac{m}{2}\log\det(\mathbf{\Sigma}) \\
&  -\frac{1}{2}\sum_{i=1}^{m}\mathbb{E}_{\boldsymbol{\gamma}_i|\mathbf{y}_i,\boldsymbol{\theta}^{(t)}} (\boldsymbol{\gamma}_i^T\mathbf{\Sigma}^{-1}\boldsymbol{\gamma}_i \mid  \mathbf{y}_i, \boldsymbol{\theta}^{(t)}) -\frac{1}{2\sigma^2}\sum_{i=1}^{m}\mathbb{E}_{\boldsymbol{\gamma}_i|\mathbf{y}_i,\boldsymbol{\theta}^{(t)}}\left[(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma}_i)^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma}_i)\mid \mathbf{y}_i,\boldsymbol{\theta}^{(t)} \right]\\
\end{aligned}
$$

Considering the two components of the expectation, we have:
$$
\begin{aligned}
& \mathbb{E}_{\boldsymbol{\gamma}_i|\mathbf{y}_i,\boldsymbol{\theta}^{(t)}} (\boldsymbol{\gamma}_i^T\mathbf{\Sigma}^{-1}\boldsymbol{\gamma}_i \mid  \mathbf{y}_i, \boldsymbol{\theta}^{(t)}) \\
=& \operatorname{tr}(\mathbf{\Sigma}^{-1}\mathbb{V}_{\boldsymbol{\gamma}_i}) + \mathbb{E}_{\boldsymbol{\gamma}_i}^T\mathbf{\Sigma}^{-1}\mathbb{E}_{\boldsymbol{\gamma}_i} \\
& \mathbb{E}_{\boldsymbol{\gamma}_i|\mathbf{y}_i,\boldsymbol{\theta}^{(t)}}\left[(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma}_i)^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}-\mathbf{Z}\boldsymbol{\gamma}_i)\mid \mathbf{y}_i,\boldsymbol{\theta}^{(t)} \right] \\
=& (\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}) - 2(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i|\mathbf{y}_i,\boldsymbol{\theta}^{(t)}}\left[\boldsymbol{\gamma}_i\mid \mathbf{y}_i,\boldsymbol{\theta}^{(t)} \right] + \mathbb{E}_{\boldsymbol{\gamma}_i|\mathbf{y}_i,\boldsymbol{\theta}^{(t)}}\left[\boldsymbol{\gamma}_i^T\mathbf{Z}^T\mathbf{Z}\boldsymbol{\gamma}_i\mid \mathbf{y}_i,\boldsymbol{\theta}^{(t)} \right] \\
=& (\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}) - 2(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i} + \operatorname{tr}(\mathbf{Z}^T\mathbf{Z}\mathbb{V}_{\boldsymbol{\gamma}_i}) + \mathbb{E}_{\boldsymbol{\gamma}_i}^T\mathbf{Z}^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i} \\
\end{aligned}
$$

Where $\mathbb{V}_{\boldsymbol{\gamma}_i}$ is the conditional covariance matrix of $\boldsymbol{\gamma}_i$ given $\mathbf{y}_i$ and $\boldsymbol{\theta}^{(t)}$ and $\mathbb{E}_{\boldsymbol{\gamma}_i}$ is the conditional expectation of $\boldsymbol{\gamma}_i$. The formula is given in Question 1 (the upper script of $\mathbf{\Sigma},\ \boldsymbol{\beta},\ \sigma$ are omitted for simplicity).:

$$
\begin{aligned}
\mathbb{V}_{\boldsymbol{\gamma}_i} &= \mathbf{\Sigma} - \mathbf{\Sigma}\mathbf{Z}^T (\mathbf{Z}\mathbf{\Sigma}\mathbf{Z}^T + \sigma^2 \mathbf{I}_n)^{-1}\mathbf{Z}\mathbf{\Sigma} \\  
\mathbb{E}_{\boldsymbol{\gamma}_i} &= \mathbf{\Sigma}\mathbf{Z}^T (\mathbf{Z}\mathbf{\Sigma}\mathbf{Z}^T + \sigma^2 \mathbf{I}_n)^{-1}(\mathbf{y}_i - \mathbf{X}_i\boldsymbol{\beta}) \\
\end{aligned}
$$

Put everything together (maybe too long to show):

$$
\begin{aligned}
& Q(\boldsymbol{\theta}|\boldsymbol{\theta}^{(t)}) \\
=& \sum_{i=1}^m\left\{
    -\frac{n_i}{2}\log(2\pi) 
    -\frac{n_i}{2}\log(\sigma^2) 
    -\frac{q}{2}\log(2\pi) 
    -\frac{1}{2}\log\det(\mathbf{\Sigma}) 
    -\frac{1}{2\sigma^2}\left[(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}) - 2(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i} + \operatorname{tr}(\mathbf{Z}^T\mathbf{Z}\mathbb{V}_{\boldsymbol{\gamma}_i}) + \mathbb{E}_{\boldsymbol{\gamma}_i}^T\mathbf{Z}^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i} 
                              \right]
    -\frac{1}{2}\left[ \operatorname{tr}(\mathbf{\Sigma}^{-1}\mathbb{V}_{\boldsymbol{\gamma}_i}) + \mathbb{E}_{\boldsymbol{\gamma}_i}^T\mathbf{\Sigma}^{-1}\mathbb{E}_{\boldsymbol{\gamma}_i}
                      \right] 
    \right\}
\end{aligned}
$$

#### Part 3
$\boldsymbol{\theta}^{(t+1)} = \operatorname{argmax}_{\boldsymbol{\theta}} Q(\boldsymbol{\theta} \mid \boldsymbol{\theta}^{(t)})$. So, we take derivative of $Q(\boldsymbol{\theta} \mid \boldsymbol{\theta}^{(t)})$ with respect to $\boldsymbol{\theta}$ and set it to zero.

For $\boldsymbol{\beta}$:
Everything containing $\boldsymbol{\beta}$ is shown in $Q_{\boldsymbol{\beta}}$:

$$
\begin{aligned}
Q_{\boldsymbol{\beta}} &= \sum_{i=1}^m \left\{
    -\frac{1}{2\sigma^2}\left[(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}) - 2(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i}
                              \right]
    \right\} \\
\frac{\partial Q}{\partial \boldsymbol{\beta}} &=
    \sum_{i=1}^m \left\{
    - \frac{1}{\sigma^2}\left[\mathbf{X}_i^T\mathbf{X}_i\boldsymbol{\beta} - \mathbf{X}_i^T\mathbf{y}_i + \mathbf{X}_i^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i} \right] \right\} \\
    &= -\frac{1}{\sigma^2} \sum_{i=1}^m \left(\mathbf{X}_i^T\mathbf{X}_i\boldsymbol{\beta} \right) + \frac{1}{\sigma^2} \sum_{i=1}^m \left(\mathbf{X}_i^T\mathbf{y}_i - \mathbf{X}_i^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i} \right) \\
\text{Thus}\quad \boldsymbol{\beta}^{(t+1)} &=  \left[\sum_{i=1}^m \left(\mathbf{X}_i^T\mathbf{X}_i\right)\right]^{-1}\sum_{i=1}^m \left(\mathbf{X}_i^T\mathbf{y}_i - \mathbf{X}_i^T\mathbf{Z}\mathbb{E}^{(t)}_{\boldsymbol{\gamma}_i} \right) 
\end{aligned} 
$$

For $\sigma^2$:

$$
\begin{aligned}
Q_{\sigma^2} &= \sum_{i=1}^m \left\{-\frac{n_i}{2}\log(\sigma^2) -\frac{1}{2\sigma^2}\left[(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}) - 2(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i} + \operatorname{tr}(\mathbf{Z}^T\mathbf{Z}\mathbb{V}_{\boldsymbol{\gamma}_i}) + \mathbb{E}_{\boldsymbol{\gamma}_i}^T\mathbf{Z}^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i}\right] \right\} \\
\frac{\partial Q}{\partial \sigma^2}& = -\frac{1}{\sigma^2} \sum_{i=1}^m \frac{n_i}{2} + \frac{1}{2(\sigma^2)^2} \sum_{i=1}^m \left[(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}) - 2(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta})^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i} + \operatorname{tr}(\mathbf{Z}^T\mathbf{Z}\mathbb{V}_{\boldsymbol{\gamma}_i}) + \mathbb{E}_{\boldsymbol{\gamma}_i}^T\mathbf{Z}^T\mathbf{Z}\mathbb{E}_{\boldsymbol{\gamma}_i}\right] \\
\text{Thus}\quad {\sigma^2}^{(t+1)} &= \frac{1}{\sum_{i=1}^m n_i} \sum_{i=1}^m \left[(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}^{(t)})^T(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}^{(t)}) - 2(\mathbf{y_i}-\mathbf{X}_i\boldsymbol{\beta}^{(t)})^T\mathbf{Z}\mathbb{E}^{(t)}_{\boldsymbol{\gamma}_i} + \operatorname{tr}(\mathbf{Z}^T\mathbf{Z}\mathbb{V}^{(t)}_{\boldsymbol{\gamma}_i}) + \mathbb{E}_{\boldsymbol{\gamma}_i}^{(t)T}\mathbf{Z}^T\mathbf{Z}\mathbb{E}^{(t)}_{\boldsymbol{\gamma}_i}\right]
\end{aligned}
$$

For $\mathbf{\Sigma}$:

$$
\begin{aligned}
Q_{\mathbf{\Sigma}} &= \sum_{i=1}^m \left\{-\frac{1}{2}\log\det(\mathbf{\Sigma}) -\frac{1}{2}\left[ \operatorname{tr}(\mathbf{\Sigma}^{-1}\mathbb{V}_{\boldsymbol{\gamma}_i}) + \mathbb{E}_{\boldsymbol{\gamma}_i}^T\mathbf{\Sigma}^{-1}\mathbb{E}_{\boldsymbol{\gamma}_i} \right] \right\} \\ 
&= \sum_{i=1}^m \left\{-\frac{1}{2}\log\det(\mathbf{\Sigma}) 
                       -\frac{1}{2}\left[ \operatorname{tr}(\mathbf{\Sigma}^{-1}\mathbb{V}_{\boldsymbol{\gamma}_i}) + \operatorname{tr}(\mathbb{E}_{\boldsymbol{\gamma}_i}^T\mathbf{\Sigma}^{-1}\mathbb{E}_{\boldsymbol{\gamma}_i})\right] \right\} \\ 
&= \sum_{i=1}^m \left\{-\frac{1}{2}\log\det(\mathbf{\Sigma}) 
                       -\frac{1}{2} \operatorname{tr}\left[ \mathbf{\Sigma}^{-1}(\mathbb{V}_{\boldsymbol{\gamma}_i} + \mathbb{E}_{\boldsymbol{\gamma}_i}\mathbb{E}_{\boldsymbol{\gamma}_i}^T)\right] \right\} \\ 
\frac{\partial Q}{\partial \mathbf{\Sigma}} &= -\frac{1}{2}\sum_{i=1}^m \left\{ \mathbf{\Sigma}^{-1} - \mathbf{\Sigma}^{-1}(\mathbb{V}_{\boldsymbol{\gamma}_i} + \mathbb{E}_{\boldsymbol{\gamma}_i}\mathbb{E}_{\boldsymbol{\gamma}_i}^T)\mathbf{\Sigma}^{-1} \right\} \\
\mathbf{\Sigma}^{(t+1)} &= \frac{1}{m} \sum_{1}^{m} \left(\mathbb{V}_{\boldsymbol{\gamma}_i}^{(t)} + \mathbb{E}_{\boldsymbol{\gamma}_i}^{(t)}\mathbb{E}_{\boldsymbol{\gamma}_i}^{(t)T}\right)
\end{aligned}
$$

## Q3. (20 pts) Objective of a single datum

We modify the code from HW6 to evaluate the objective, the conditional mean of $\boldsymbol{\gamma}$, and the conditional variance of $\boldsymbol{\gamma}$. Start-up code is provided below. You do _not_ have to use this code.

In [80]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # posterior mean and variance of random effects γ
    μγ         :: Vector{T} # posterior mean of random effects
    νγ         :: Matrix{T} # posterior variance of random effects
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    rtr        :: Vector{T}
    xty        :: Vector{T}
    zty        :: Vector{T}
    ztr        :: Vector{T}
    ltztr      :: Vector{T}
    xtr        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ltztzl     :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2:: Matrix{T}
    storage_qq3:: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
    y::Vector{T}, 
    X::Matrix{T}, 
    Z::Matrix{T}) where T <: AbstractFloat
    n, p, q = size(X, 1), size(X, 2), size(Z, 2)
    μγ         = Vector{T}(undef, q)
    νγ         = Matrix{T}(undef, q, q)
    yty        = abs2(norm(y))
    rtr        = Vector{T}(undef, 1)
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    ztr        = similar(zty)
    ltztr      = similar(zty)
    xtr        = Vector{T}(undef, p)
    storage_p  = similar(xtr)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = similar(storage_q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ltztzl     = similar(ztz)
    storage_qq = similar(ztz)
    storage_qq2= similar(ztz)
    storage_qq3= similar(ztz)
    LmmObs(y, X, Z, μγ, νγ, 
        yty, rtr, xty, zty, ztr, ltztr, xtr,
        storage_p, storage_q, storage_q2,
        xtx, ztx, ztz, ltztzl, storage_qq, storage_qq2, storage_qq3)
end

"""
    logl!(obs::LmmObs, β, Σ, L, σ², updater = false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `Σ`, 
and `σ²`. The lower triangular Cholesky factor `L` of `Σ` must be supplied too.
The fields `obs.μγ` and `obs.νγ` are overwritten by the posterior mean and 
posterior variance of random effects. If `updater==true`, fields `obs.ztr`, 
`obs.xtr`, and `obs.rtr` are updated according to input parameter values. 
Otherwise, it assumes these three fields are pre-computed. 
"""
function logl!(
        obs     :: LmmObs{T}, 
        β       :: Vector{T}, 
        Σ       :: Matrix{T},
        L       :: Matrix{T},
        σ²      :: T,
        updater :: Bool = false
        ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    σ²inv   = inv(σ²)
    ####################
    # Evaluate objective
    ####################
    # form the q-by-q matrix: Lt Zt Z L
    copy!(obs.ltztzl, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.ltztzl) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.ltztzl) # O(q^3)        
    # form the q-by-q matrix: M = σ² I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ltztzl)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # Zt * res
    updater && BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.ztr, obs.zty)) # O(pq)
    # Lt * (Zt * res)
    BLAS.trmv!('L', 'T', 'N', L, copy!(obs.ltztr, obs.ztr))    # O(q^2)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, copy!(obs.storage_q, obs.ltztr)) # O(q^3)
    # Xt * res = Xt * y - Xt * X * β
    updater && BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), copy!(obs.xtr, obs.xty))
    # l2 norm of residual vector
    updater && (obs.rtr[1] = obs.yty - dot(obs.xty, β) - dot(obs.xtr, β))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q # log det term
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (obs.rtr[1] - qf) * σ²inv 
    logl /= -2
    ######################################
    # Evaluate posterior mean and variance
    ######################################    
    # posterior variance
    if false     # form 1 causing monotonicity issue
        copy!(obs.storage_qq2, L)
        BLAS.trsm!('R', 'U', 'N', 'N', T(1), obs.storage_qq, obs.storage_qq2)  # L Chol(M)^-1
        mul!(obs.storage_qq3, obs.ztz, obs.storage_qq2)  # Z'ZL Chol(M)^-1
        mul!(obs.storage_qq2, Σ, obs.storage_qq3)  # ΣZ'ZL Chol(M)^-1
        mul!(obs.storage_qq3, obs.storage_qq2, obs.storage_qq2')  # ΣZ'ZLM^-1L'Z'ZΣ
        copy!(obs.νγ, obs.storage_qq3)
        obs.νγ ./= σ²
        mul!(obs.storage_qq2, obs.ztz, Σ)
        mul!(obs.storage_qq3, Σ, obs.storage_qq2)  # ΣZ'ZΣ
        obs.storage_qq3 ./= σ²
        obs.νγ .-= obs.storage_qq3
        obs.νγ .+= Σ
    end 
    
    if true     # form 2 works well
        copy!(obs.storage_qq2, L)
        LAPACK.trtri!('L', 'N', obs.storage_qq2) # inverse triangular L^-1
        mul!(obs.storage_qq3, 
             UpperTriangular(obs.storage_qq2'),
             LowerTriangular(obs.storage_qq2))   # Σ^-1
        obs.storage_qq2 .= 1 / σ² .* obs.ztz .+ obs.storage_qq3 # σ^2Z'Z + σ^2Σ^-1
        
        LAPACK.potrf!('U', obs.storage_qq2)  # Cholesky decompsion
        # obs.storage_qq3 .= I(q)  # cause memory allocation!!
        # BLAS.trsm!('R', 'U', 'N', 'N', T(1), obs.storage_qq2, obs.storage_qq3) 
        # mul!(obs.νγ, obs.storage_qq3, obs.storage_qq3')
        LAPACK.potri!('U', obs.storage_qq2)  # inverse of PD (upper triangular)
        obs.νγ .= obs.storage_qq2
        @inbounds for i in 1:q, j in 1:i
            obs.νγ[i, j] = obs.νγ[j, i]
        end
    end
    
    # posterior mean
    # E = σ^-2 (VZ'y - VZ'Xβ)
    mul!(obs.μγ, obs.νγ, obs.zty)
    mul!(obs.storage_q, obs.ztx, β)
    mul!(obs.storage_q2, obs.νγ, obs.storage_q)
    obs.μγ .-= obs.storage_q2
    obs.μγ ./= σ²
    
    ###################
    # Return
    ###################        
    return logl
end


logl!

It is a good idea to test correctness and efficiency of the single datum objective/posterior mean/var evaluator here. It's the same test datum as in HW3 and HW6.

In [81]:
Random.seed!(257)

# dimension
n, p, q = 2000, 5, 3
# predictors
X = [ones(n) randn(n, p - 1)]
Z = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [82]:
@show logl = logl!(obs, β, Σ, L, σ², true)
@show obs.μγ
@show obs.νγ;

logl = logl!(obs, β, Σ, L, σ², true) = -3256.1793358058376
obs.μγ = [0.10608689301332763, -0.2510419060257252, -1.465397940985037]
obs.νγ = [0.0007494356395786613 -1.2183420464422369e-6 -2.176783682941746e-6; -1.2183420464422369e-6 0.0007542331466357765 2.1553464612468773e-5; -2.176783682941746e-6 2.1553464612468773e-5 0.0007465271344917232]


You will lose all 20 points if following statement throws `AssertionError`.

In [83]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.μγ - [0.10608689301333621, 
        -0.25104190602577225, -1.4653979409855415]) < 1e-4
@assert norm(obs.νγ - [
        0.0007494356395909563 -1.2183420093769967e-6 -2.176783643112221e-6; 
        -1.2183420282298223e-6 0.0007542331467601107 2.1553464632686345e-5; 
        -2.1767836636008638e-6 2.1553464641863096e-5 0.0007465271342535443
        ]) < 1e-4

### Efficiency

Benchmark for efficiency.

In [85]:
bm_obj = @benchmark logl!($obs, $β, $Σ, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.083 μs …   9.613 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.167 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.176 μs ± 176.752 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

        ▁▅ ▁   ▂ ▃▆ ▂▂  ▃ ▃█ ▁▄                                
  ▁▁▂▃▇▅██▇██▄▅█▆█████▅▇█▇██▇██▅▇█▄▅▆▄▃▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.08 μs         Histogram: frequency by time        1.33 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median runt time is 800ns. You will get full credit if the median run time is within 10μs. The points you will get are

In [86]:
clamp(10 / (median(bm_obj).time / 1e3) * 10, 0, 10)

10.0

In [30]:
# # check for type stability
# @code_warntype logl!(obs, β, Σ, L, σ²)

In [ ]:
# using Profile

# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, Σ, L, σ²); end
# Profile.print(format=:flat)

## Q4. LmmModel type

We modify the `LmmModel` type in HW6 to hold all data points, model parameters, and intermediate arrays.

In [87]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat}
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β      :: Vector{T}
    Σ      :: Matrix{T}
    L      :: Matrix{T}
    σ²     :: Vector{T}    
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty    :: Vector{T}
    xtr    :: Vector{T}
    ztr2   :: Vector{T}
    xtxinv :: Matrix{T}
    ztz2   :: Matrix{T}
    storage_p :: Vector{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p      = size(obsvec[1].X, 2)
    q      = size(obsvec[1].Z, 2)
    # parameters
    β      = Vector{T}(undef, p)
    Σ      = Matrix{T}(undef, q, q)
    L      = Matrix{T}(undef, q, q)
    σ²     = Vector{T}(undef, 1)    
    # intermediate arrays
    xty    = zeros(T, p)
    xtr    = similar(xty)
    ztr2   = Vector{T}(undef, abs2(q))
    xtxinv = zeros(T, p, p)
    storage_p = Vector{T}(undef, p)
    # pre-calculate \sum_i Xi^T Xi and \sum_i Xi^T y_i
    @inbounds for i in eachindex(obsvec)
        obs = obsvec[i]
        BLAS.axpy!(T(1), obs.xtx, xtxinv)
        BLAS.axpy!(T(1), obs.xty, xty)
    end
    # invert X'X
    LAPACK.potrf!('U', xtxinv)
    LAPACK.potri!('U', xtxinv)
    LinearAlgebra.copytri!(xtxinv, 'U')
    ztz2   = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, Σ, L, σ², xty, xtr, ztr2, xtxinv, ztz2, 
             storage_p)
end

LmmModel

## Q5. Implement EM update

Let's write the key function `update_em!` that performs one iteration of EM update.

In [88]:
"""
    update_em!(m::LmmModel, updater::Bool = false)

Perform one iteration of EM update. It returns the log-likelihood calculated 
from input `m.β`, `m.Σ`, `m.L`, and `m.σ²`. These fields are then overwritten 
by the next EM iterate. The fields `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i]` are updated according to the resultant `m.β`. If `updater==true`, 
the function first updates `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` according to `m.β`. If `updater==false`, it assumes these fields 
are pre-computed.
"""
function update_em!(m::LmmModel{T}, updater::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    # calculate log-likelihood
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.Σ, m.L, m.σ²[1], updater)
    end
    
    # TODO: update m.β  
    fill!(m.storage_p, zero(T))
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        mul!(obs.storage_p, obs.ztx', obs.μγ)  # Zt * μγ
        BLAS.axpy!(T(1), obs.storage_p, m.storage_p) # sum XtZ * μγ
    end
    m.storage_p .= m.xty .- m.storage_p
    mul!(m.β, m.xtxinv, m.storage_p) 
    
    # TODO: update m.data[i].ztr, m.data[i].xtr, m.data[i].rtr
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        # Zt * res
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.ztr, obs.zty)) 
        # Xt * res = Xt * y - Xt * X * β
        BLAS.gemv!('N', T(-1), obs.xtx, m.β, T(1), copy!(obs.xtr, obs.xty))
        # l2 norm of residual vector
        obs.rtr[1] = obs.yty - dot(obs.xty, m.β) - dot(obs.xtr, m.β)
    end
    
    # TODO: update m.σ²
    m.σ²[1] = zero(T)
    N = zero(T)
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        N += size(obs.X, 1)  # total sample size
        mul!(obs.storage_qq, obs.ztz, obs.νγ)  # Z'ZV
        m.σ²[1] += tr(obs.storage_qq)  # tr(Z'ZV)
        mul!(obs.storage_q, obs.ztz, obs.μγ)  # Z'Zμ
        m.σ²[1] += dot(obs.storage_q, obs.μγ)  # μ'Z'Zμ
        m.σ²[1] -= 2 * dot(obs.ztr, obs.μγ)  # -2r'Zμ
        m.σ²[1] += obs.rtr[1]
    end
    m.σ²[1] /= N
    
    # update m.Σ and m.L
    fill!(m.Σ, zero(T))
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        mul!(obs.storage_qq, obs.μγ, obs.μγ')  # μμ'
        obs.storage_qq .+= obs.νγ  # μμ' + V
        m.Σ .+= obs.storage_qq
    end
    m.Σ ./= length(m.data)
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    m.L .= LowerTriangular(m.L)
    # m.L .= Matrix(cholesky(Symmetric(m.Σ)).L) # not work for some reason
    
    logl
end

update_em!

## Q6. (30 pts) Test data

Let's generate a synthetic longitudinal data set (same as HW6) to test our algorithm.

In [89]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

### Correctness

Evaluate log-likelihood and gradient at the true parameter values.

In [90]:
copy!(lmm.β, βtrue)
copy!(lmm.Σ, Σtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj1 = update_em!(lmm, true)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²
println()
@show obj2 = update_em!(lmm, false)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

obj1 = update_em!(lmm, true) = -2.8400684383699736e6
lmm.β = [0.10003613673625013, 6.500382871080181, -3.499864634211224, 0.9997124657606651, 4.999230851463544]
lmm.Σ = [1.9903882760455316 0.06862095707037434 0.053472901794726614; 0.06862095707037434 1.2813220461216903 -0.09044913324906328; 0.053472901794726614 -0.09044913324906328 0.9435400745724166]
lmm.L = [1.4108112120498375 0.0 0.0; 0.04863936186803587 1.1309094829378523 0.0; 0.037902237619045556 -0.0816092492747239 0.9671832429217857]
lmm.σ² = [1.4987367279243704]

obj2 = update_em!(lmm, false) = -2.8400604605420595e6
lmm.β = [0.10007136571174427, 6.500383550631511, -3.4998642980415435, 0.9997119269497639, 4.999229480978786]
lmm.Σ = [1.9903775380783681 0.06870107683947126 0.05354351750443929; 0.06870107683947126 1.281440921371172 -0.09059223997463756; 0.05354351750443929 -0.09059223997463756 0.9434431688085722]
lmm.L = [1.4108074064443978 0.0 0.0; 0.04869628308276029 1.1309595896339955 0.0; 0.037952393260666885 -0.081736236473703

1-element Vector{Float64}:
 1.498734554416028

Test correctness. You will loss all 30 points if following code throws `AssertError`.

In [91]:
@assert abs(obj1 - (-2.840068438369969e6)) < 1e-4
@assert abs(obj2 - (-2.84006046054206e6)) < 1e-4

### Efficiency

Test efficiency of EM update.

In [92]:
bm_emupdate = @benchmark update_em!($lmm, true) setup=(
    copy!(lmm.β, βtrue);
    copy!(lmm.Σ, Σtrue);
    copy!(lmm.L, Ltrue);
    lmm.σ²[1] = σ²true)

BenchmarkTools.Trial: 3326 samples with 1 evaluation.
 Range (min … max):  1.456 ms …  3.030 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.490 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.502 ms ± 53.940 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▄▆█▇▅▃▂▁                                                 
  ▃▅█████████▇▆▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂ ▃
  1.46 ms        Histogram: frequency by time        1.73 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median run time is 1ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [93]:
clamp(10 / (median(bm_emupdate).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [94]:
clamp(10 - median(bm_emupdate).memory / 100, 0, 10)

10.0

## Q7. Starting point

We use the same least squares estimates as in HW6 as starting point. 

In [95]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # LS estimate for β
    mul!(m.β, m.xtxinv, m.xty)
    # LS etimate for σ2 and Σ
    rss, ntotal = zero(T), 0
    fill!(m.ztz2, 0)
    fill!(m.ztr2, 0)    
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        ntotal += length(obs.y)
        # update Xt * res
        BLAS.gemv!('N', T(-1), obs.xtx, m.β, T(1), copy!(obs.xtr, obs.xty))
        # rss of i-th individual
        rss += obs.yty - dot(obs.xty, m.β) - dot(obs.xtr, m.β)
        # update Zi' * res
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.ztr, obs.zty))
        # Zi'Zi ⊗ Zi'Zi
        kron_axpy!(obs.ztz, obs.ztz, m.ztz2)
        # Zi'res ⊗ Zi'res
        kron_axpy!(obs.ztr, obs.ztr, m.ztr2)
    end
    m.σ²[1] = rss / ntotal
    # LS estimate for Σ = LLt
    LAPACK.potrf!('U', m.ztz2)
    BLAS.trsv!('U', 'T', 'N', m.ztz2, m.ztr2)
    BLAS.trsv!('U', 'N', 'N', m.ztz2, m.ztr2)
    copyto!(m.Σ, m.ztr2)
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    for j in 2:q, i in 1:j-1
        m.L[i, j] = 0
    end
    m
end

"""
    kron_axpy!(A, X, Y)

Overwrite `Y` with `A ⊗ X + Y`. Same as `Y += kron(A, X)` but
more memory efficient.
"""
function kron_axpy!(
        A::AbstractVecOrMat{T},
        X::AbstractVecOrMat{T},
        Y::AbstractVecOrMat{T}
        ) where T <: Real
    m, n = size(A, 1), size(A, 2)
    p, q = size(X, 1), size(X, 2)
    @assert size(Y, 1) == m * p
    @assert size(Y, 2) == n * q
    @inbounds for j in 1:n
        coffset = (j - 1) * q
        for i in 1:m
            a = A[i, j]
            roffset = (i - 1) * p            
            for l in 1:q
                r = roffset + 1
                c = coffset + l
                for k in 1:p                
                    Y[r, c] += a * X[k, l]
                    r += 1
                end
            end
        end
    end
    Y
end

kron_axpy!

In [96]:
init_ls!(lmm)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

lmm.β = [0.18207934611476326, 6.50048070099372, -3.4979107842091595, 1.0011132962297953, 5.0002519857919285]
lmm.Σ = [1.9794302836685058 0.07258461003916691 0.05717147035274022; 0.07258461003916693 1.2840385734767714 -0.07707942768978564; 0.05717147035274022 -0.07707942768978564 0.9509885905046899]
lmm.L = [1.4069222734993236 0.0 0.0; 0.05159105901325531 1.1319792118703693 0.0; 0.04063584138912114 -0.06994463586493145 0.9718256360134827]
lmm.σ² = [5.709004733413668]


1-element Vector{Float64}:
 5.709004733413668

## Q8. Estimation by EM

We write a function `fit!` that implements the EM algorithm for estimating LMM.

In [97]:
"""
    fit!(m::LmmModel)

Fit an `LmmModel` object by MLE using a EM algorithm. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m       :: LmmModel;
        maxiter :: Integer       = 10_000,
        ftolrel :: AbstractFloat = 1e-12,
        prtfreq :: Integer       = 0
    )
    obj = update_em!(m, true)
    for iter in 0:maxiter
        obj_old = obj
        # EM update
        obj = update_em!(m, false)
        # print obj
        prtfreq > 0 && rem(iter, prtfreq) == 0 && println("iter=$iter, obj=$obj")
        # check monotonicity
        obj < obj_old && (@warn "monotoniciy violated")
        # check convergence criterion
        (obj - obj_old) < ftolrel * (abs(obj_old) + 1) && break
        # warning about non-convergence
        iter == maxiter && (@warn "maximum iterations reached")
    end
    m
end


fit!

## Q9. (20 pts) Test drive

Now we can run our EM algorithm to compute the MLE.

In [98]:
# initialize from least squares
init_ls!(lmm)

@time fit!(lmm, ftolrel = 1e-12, prtfreq = 1);

println("objective value at solution: ", update_em!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)

iter=0, obj=-2.8400688821783797e6
iter=1, obj=-2.8400587867556894e6
iter=2, obj=-2.840058786725548e6
iter=3, obj=-2.840058786725478e6
  0.025193 seconds (17.18 k allocations: 1.211 MiB, 69.26% compilation time)


objective value at solution: -2.8400587867254117e6

solution values:
lmm.β = [0.18207855803149206, 6.500383547392576, -3.499864296224195, 0.9997119252318636, 4.999229481214088]
lmm.σ² = [1.498734551919432]
lmm.L * transpose(lmm.L) = [1.983631984277848 0.06575202041811087 0.05528837687885904; 0.06575202041811087 1.2814409879932709 -0.09059254193517799; 0.05528837687885904 -0.09059254193517799 0.9434430471656207]


3×3 Matrix{Float64}:
 1.98363     0.065752    0.0552884
 0.065752    1.28144    -0.0905925
 0.0552884  -0.0905925   0.943443

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [99]:
# objective at solution should be close enough to the optimal
@assert update_em!(lmm) > -2.840059e6

### Efficiency

My median run time 5ms. You get 10 points if your median run time is within 1s.

In [100]:
bm_em = @benchmark fit!($lmm) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 674 samples with 1 evaluation.
 Range (min … max):  6.884 ms …   8.460 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.067 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.145 ms ± 225.059 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▁▄▅█▆█▂▃▂                                                 
  ▄▃▇███████████▆▆▆▅▄▆▄▅▅▃▃▅▅▅▄▃▃▄▃▃▄▂▁▃▃▁▂▃▃▃▃▃▂▃▃▃▂▃▃▃▂▁▁▂▃ ▄
  6.88 ms         Histogram: frequency by time        7.93 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [101]:
# this is the points you get
clamp(1 / (median(bm_em).time / 1e9) * 10, 0, 10)

10.0

## Q10. (10 pts) EM vs Newton type algorithms

Contrast EM algorithm to the Newton type algorithms (gradient free, gradient based, using Hessian) in HW6, in terms of the stability, convergence rate (how fast the algorithm is converging),  final objective value, total run time, derivation, and implementation efforts. 

### Q10 Answer
Comparing the results from last homework, EM algorithm is quicker in total run time and convergence rate than Newton type algorithms. They got similar final results with a objective function $\approx -2840059$. The Mathematical derivation for EM is easier than Newton type algorithms (Newton type algorithms require complex matrix derivatives). However, for the coding part, my personal feeling is that EM is more difficult than Newton type algorithms because we have handy solvers for the Newton type algorithm, which saved us a lot of effort. 